In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("C:/Users/Harvey/Desktop/Yelp_data_set/restuarant_review_balanced.csv")

In [3]:
df.head()

,sentiment,stars,Processed_Reviews
0,0,1,place go hill clearly cut back staff food qual...
1,0,1,walk around friday afternoon sit table bar wal...
2,0,1,cannot believe things change years pick duck c...
3,1,5,cant really find anything wrong place pastas p...
4,1,4,great lunch today staff helpful assist selecti...


In [4]:
df.Processed_Reviews.apply(lambda x: len(x.split(" "))).mean()

58.18184846606756

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [25]:
train = df.sample(frac = 0.8,random_state = 200)
test = df.drop(train.index)

,sentiment,stars,Processed_Reviews
151246,0,2,much better chicken place go days theyre slow ...
87000,0,1,place overrate burger dry stingy gravy poutine...
102233,1,5,bbq probably favorite food give usually pricey...
254520,0,1,food service restaurant leave much desire arri...
135806,0,2,chicken red curry lunch special piece chicken ...


In [26]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train['Processed_Reviews'])
list_tokenized_train = tokenizer.texts_to_sequences(train['Processed_Reviews'])

maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train['sentiment']

In [28]:
maxlen = 130
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
y = train['sentiment']

embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 100
epochs = 3
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 165222 samples, validate on 41306 samples
Epoch 1/3
165222/165222 [==============================] - 316s 2ms/step - loss: 0.1802 - acc: 0.9273 - val_loss: 0.1451 - val_acc: 0.9438
Epoch 2/3
165222/165222 [==============================] - 310s 2ms/step - loss: 0.1235 - acc: 0.9531 - val_loss: 0.1387 - val_acc: 0.9483
Epoch 3/3
165222/165222 [==============================] - 304s 2ms/step - loss: 0.0990 - acc: 0.9634 - val_loss: 0.1411 - val_acc: 0.9484


In [30]:
y_test = test["sentiment"]
list_sentences_test = test['Processed_Reviews']
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, confusion_matrix
print('accuracy :{0}'.format(accuracy_score(y_pred, y_test)))
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
#confusion_matrix(y_pred, y_test)

accuracy :0.9473194917880384
F1-score: 0.950159416571994
Confusion matrix:


In [22]:
####################### made up review 
test = "I can't love this place "
test2 = "the food is not bad really nice place"
test_set = [test,test2]
y_test = [1,1]
list_tokenized_train = tokenizer.texts_to_sequences(test_set)

maxlen = 130
X_te = pad_sequences(list_tokenized_train, maxlen=maxlen)
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)

In [23]:
y_pred

array([[ True],
       [False]])